In [ ]:

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, resnet101, inception_v3,resnet18
import time
import argparse
import math
import json
import pickle
import numpy as np
from PIL import ImageFile
from torch.utils.data.sampler import SubsetRandomSampler

ImageFile.LOAD_TRUNCATED_IMAGES = True

epochs = 100

lr_init=0.0001
batch_size=32
num_classes=5


In [ ]:
# Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                      transforms.RandomRotation(degrees=15),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.CenterCrop(size=224),  # Image net standards
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(224),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

In [ ]:
# Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                      transforms.RandomRotation(degrees=15),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.CenterCrop(size=224),  # Image net standards
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(224),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

In [ ]:
train_dir='/content/drive/My Drive/fuzzydataset0/train'
test_dir='/content/drive/My Drive/dataset8/test'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
#Loading in the dataset

train_data = datasets.ImageFolder(train_dir,transform=train_transforms)
test_data = datasets.ImageFolder(test_dir,transform=test_transforms)

print(train_data.class_to_idx)
# number of subprocesses to use for data loading
num_workers = 0
# percentage of training set to use as validation
valid_size = 0.2

test_size = 0.0

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
test_indices=list(range(len(test_data)))
# print(indices)
np.random.shuffle(indices)
# print(indices)
valid_split = int(np.floor((valid_size) * num_train))
test_split = int(np.floor((valid_size+test_size) * num_train))
valid_idx, test_idx, train_idx = indices[:valid_split], indices[valid_split:test_split], indices[test_split:]
print(len(valid_idx), len(test_indices), len(train_idx))

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_indices)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    sampler=test_sampler, num_workers=num_workers)


{'martensite': 0, 'pearlite': 1, 'spheroidite': 2}
353 221 1413


In [ ]:
model1 = models.resnet18(pretrained=False)
model1
fc = nn.Sequential(
    nn.Linear(512, 320),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(320,5)
)
model1.fc = fc

In [ ]:
model2 = models.alexnet(pretrained=False)
model2
fc = nn.Sequential(
    nn.Linear(9216, 4096),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(4096,5),
    
)
model2.classifier = fc




In [ ]:
# model_save_name = "resnet101_5class_l2.pth"
# model_save_name = "resnet101_5class_l2_sp.pth"
model_save_name1 = "resnet_5class.pth"
model_save_name2 = "alexnet_5class.pth"


path1 = F"/content/drive/My Drive/{model_save_name1}" 
path2 = F"/content/drive/My Drive/{model_save_name2}"
model1.load_state_dict(torch.load(path1))
model2.load_state_dict(torch.load(path2))
model1.to(device)
model2.to(device)



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, 

In [ ]:
criterion = nn.CrossEntropyLoss()


In [ ]:
def test(model1,model2, criterion):
  # monitor test loss and accuracy
  import numpy as np
  ori_idx=[]
  nb_classes=5
  confusion_matrix1 = torch.zeros(nb_classes, nb_classes)
  y_ori=[]
  y_pre1=[]
  y_pre2=[]
  y_prob1=[]
  y_prob2=[]
  test_loss = 0.
  correct = 0.
  total = 0.
  model1.eval()
  model2.eval()
  for batch_idx, (data, target) in enumerate(test_loader):
    # move to GPU
    if torch.cuda.is_available():
      data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output1 = model1(data)
    output2 = model2(data)
    # print(output)
    # p = torch.nn.functional.softmax(output, dim=1)
    p1 = torch.nn.functional.softmax(output1, dim=1)
    p2 = torch.nn.functional.softmax(output2, dim=1)
    # print(p1.tolist())
    # print(p2.tolist())
    ori_idx.append(target)
    
    # calculate the loss
    loss1 = criterion(output1, target)
    loss2 = criterion(output2, target)
    # update average test loss 
    test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss1.data - test_loss))
    # convert output probabilities to predicted class
    _, preds1 = torch.max(output1, 1)
    pred1 = output1.data.max(1, keepdim=True)[1]
    _, preds2 = torch.max(output2, 1)
    pred2 = output2.data.max(1, keepdim=True)[1]
    np_target=target.cpu().numpy()
    np_preds1=preds1.cpu().numpy()
    np_preds2=preds2.cpu().numpy()
    print(np_target)
    print(np_preds1)
    print(np_preds2)
    y_ori.extend(list(np_target))
    y_pre1.extend(list(np_preds1))
    y_pre2.extend(list(np_preds2))
    y_prob1.extend(list(p1.tolist()))
    y_prob2.extend(list(p2.tolist()))
   
            
  
  
  
  # print(ori_idx)
  print("actual label",y_ori)
  print("predicted label1",y_pre1)
  print("predicted label2",y_pre2)
  print("prob1 ",y_prob1)
  print("prob2 ",y_prob2)
test(model1,model2, criterion)


[1 3 0 3 1 0 4 1 1 3 3 4 1 3 1 4 3 2 3 3 3 1 2 0 3 3 3 1 1 0 0 0]
[1 3 0 3 1 0 4 1 1 3 3 4 1 3 1 3 3 2 3 3 3 1 2 0 3 3 3 1 1 0 0 0]
[1 3 2 3 1 0 4 1 1 3 3 4 1 3 1 4 3 2 3 3 3 1 2 0 3 3 3 1 1 0 0 0]
[3 0 0 1 1 4 1 0 0 4 3 4 3 3 1 3 0 2 2 0 1 4 3 2 1 4 3 3 4 0 2 1]
[3 0 0 1 1 4 1 0 0 4 3 4 3 3 1 3 0 2 2 0 1 4 3 4 1 4 3 3 4 0 2 1]
[3 0 0 3 1 4 1 0 0 4 3 4 3 3 1 3 0 2 2 0 1 4 3 3 1 4 3 3 4 0 2 1]
[3 3 1 1 3 0 0 3 3 0 3 0 2 3 3 4 1 0 0 4 1 3 0 0 0 4 3 3 1 3 0 0]
[3 3 1 1 3 0 0 3 4 0 3 0 2 3 3 4 1 0 0 4 1 4 0 0 0 4 3 3 1 3 0 0]
[3 3 1 1 3 0 0 3 3 0 3 0 2 4 3 4 1 0 0 4 1 4 0 0 0 4 4 3 1 3 0 0]
[3 3 3 3 4 0 1 4 2 3 1 0 4 3 2 3 3 2 4 3 0 2 0 1 0 1 3 2 0 2 3 2]
[3 4 3 3 4 0 1 4 2 3 1 0 3 3 2 3 3 2 2 3 0 2 0 1 0 1 3 2 0 2 3 2]
[3 4 4 3 4 0 4 4 2 3 1 0 4 3 2 3 3 2 4 3 0 2 0 1 0 1 3 2 0 2 3 2]
[1 3 1 1 3 2 3 1 3 2 3 3 3 4 4 3 2 3 0 2 3 2 3 2 0 4 3 3 1 0 1 1]
[1 4 1 1 3 2 3 1 3 2 3 3 3 3 4 4 2 3 0 2 3 2 3 2 0 4 3 3 1 0 1 1]
[1 3 1 1 3 2 3 1 3 3 3 3 3 4 4 4 2 3 2 2 3 2 3 2 0 4 3 3 1 0 1 1]
[0 4 3 2 4

In [ ]:
y_ori

NameError: ignored